# After years of thought it's finally here

In [ ]:
import pandas as pd
import numpy as np

In [219]:
df=pd.read_excel('Pokemon.xlsx',)#dtype={'Atk': 'Int64', 'SpA': 'Int64'}) # I want the atk and spa columns to be integers, not floats like 92.0
# I guess I'll modify while processing instead of while loading.

The following code merges all move columns into one column, and then drops duplicates to get a proper list of moves

In [220]:
#moveslist=pd.concat([df['Move1'],df['Move2'],df['Move3'],df['Move4'],df['Move5'],df['Move6'],df['Move7'],df['Move8'],df['Move9'],df['Move10'],df['Move11'],df['Move12'],df['Move13'],df['Move14'],df['Move15'],df['Move16'],df['Move17'],df['Move18'],df['Move19'],df['Move20'],df['Move21'],df['Move22'],df['Move23'],df['Move24'],df['Move25'],df['Move26'],df['Move27'],df['Move28'],df['Move29'],df['Move30'],df['Move31'],df['Move32'],df['Move33'],df['Move34'],df['Move35'],df['Move36']], ignore_index=True)
# straight up bad code


#moveslist=pd.concat([col for col in movesdf], axis = 0, ignore_index=True)
#this line of code doesn't work because col is just the name of the column, not the contents of the column.
#it turns out that if you try to iterate an element over a dataframe, you just get the column names.
#so you have to do df[col] for col in df.

moveslist=pd.concat([df[col] for col in df.columns if col.startswith('Move')], axis = 0, ignore_index=True)
#concat selects columns one by one and concatenates their contents vertically
# although the above code is more understandable, I could have replaced df.columns with just df. iterating over a dataframe just means iterating over its column names.

print(len(moveslist))

moveslist=moveslist.dropna() # the only way columns wouldn't have empty cells is if every pokemon had 43 moves


print(len(moveslist)) #this is 20984 before dropping empty cells, and 6728 after dropping. (20984 is 488 pokemon * 43 max moves)
print(moveslist.nunique()) #this is 213 (there are 213 unique moves)
#even after this step, I haven't converted moveslist to the unique moveslist, because I need the duplicates later for when I merge with types.

20984
6737
213


A single column with 6728 moves has been created. We now create a single column with 6728 types, merge them, and then drop duplicates.

In [221]:
typeslist=pd.concat([df[col] for col in df.columns if col.startswith('Type')], axis = 0, ignore_index=True) # the stab types do not start with 'Type'
# so this ensures that only the move type columns are included.

typeslist=typeslist.dropna()
len(typeslist) #this is 20984 without dropping empty cells, and 6729 after dropping them

6737

In [222]:
#if not isinstance(moveslist,pd.DataFrame):
    #movetable=moveslist.to_frame(name='Move')

long_movetable=moveslist.to_frame(name='Move')      #When converting a series to dataframe, if you don't name the column, it receives the name '0'.
long_movetable['Type']=typeslist               #This second column lines up perfectly with the Move column.

#movetable=movetable[:-1] #there's something extremely weird with the last row, so I'm slicing it off
# okay it appears I had some error with the last row when I first coded this, but this error doesn't reappear and I can't replicate it. So the previous line has no significance.

movetable=(long_movetable.drop_duplicates()
            .sort_values(by='Move')
            #.reset_index()             #This is so that the numbers which are in random order (because the moves are sorted) get fixed.
            #.drop(columns='index')    
            .set_index('Move'))    #Setting something else as index makes the numerical index vanish anyway, so the previous two lines are useless.

#movetable

In [223]:
movetable[['Power','Category','Accuracy','Two_Turn','Slow', 'Multi_Strike','Sheer Force Boosted','Contact','Pulse','Sound','Priority']] = [np.nan,None,np.nan,None,None,None,None,None,None,None,None]
# this accuracy column is for the numerical values of accuracy.

# slow means moves like revenge, avalanche (even if avalanche isn't guaranteed to go second)
# maybe slow would require low speed? So maybe one day I'll add a speed column.
# although tbh, RU mons are already slow anyway lol.


# 'Slicing','Bite','Punch', and the regular 'Plain_Contact' will be categories of Contact

#movetable['Contact']=movetable['Contact'].astype('category')
# when using astype to make it category, the categories are automatically derived from the values already present in the column. Here it's just 0. 
# Yes it's better to fill the column before converting it to a category, so that this automatic categorization happens.
# WHICH IS WHY I'VE DECIDED TO CONVERT THE COLUMNS INTO CATEGORIES AFTER FILLING THEM ALL.

# But if you don't want to do that, one alternative is using add_categories before inputting the actual category strings into the column.

# another alternative is to avoid astype(), and from the start use pd.Categorical(df['column'], categories=[list of categories], ordered=False).

#movetable.info()

In [224]:
# movetable[['Category','Effect']].loc['Acrobatics']=[2,3]
# The above code uses chained assignment which triggers a chainedassignmenterror warning. It's better to use loc the proper way, filtering columns as well.
# Like this:
#movetable.loc['Acrobatics',['Category','Effect']]=[2,3]
#movetable

We should either assign values for tough claws boosted moves (contact moves) either before the following physicalities like slicing, biting, 
or do it after for the remaining contact moves (I'm leaning towards the latter option)

Probably something like, "Plain_contact"

In [225]:
# okay, I will fill in the data one move group at a time.
# First let me try Sharpness (Slicing)

movetable.loc[['Aerial Ace','Air Slash', 'Air Cutter','Aqua Cutter', 'Cross Poison','Fury Cutter','Leaf Blade','Night Slash','Razor Shell','Sacred Sword','Solar Blade','X Scissor'],'Contact']='Slicing'
# need to add ceaseless edge, stone axe, psycho cut if one day a non stab mon learns these moves
# need to add a bunch of moves if one day hisuian samurott drops to RU. (Or if we include higher tiers)

Bulbapedia might sometimes not consider moves that a mon like gallade doesn't learn anymore. Make sure to include such moves for sharpness (and other abilities like biting, pulse)

In [226]:
movetable.loc[['Bullet Punch','Drain Punch','Dynamic Punch','Fire Punch','Hammer Arm','Ice Punch','Mach Punch','Thunder Punch'],'Contact']='Punch'

# a few punch moves I haven't included are
# Ice hammer, double iron bash, normal type moves, Shadow punch, sky uppercut, focus punch
# and punch moves not learned by iron fist mons are headlong rush, jet punch, meteor mash, plasma fists, rage fist, surging strikes, wicked blow.


In [227]:
movetable.loc[['Bite','Crunch','Fire Fang','Ice Fang','Jaw Lock','Poison Fang','Psychic Fangs','Thunder Fang'],'Contact']='Biting'
# haven't included hyper fang (cus normal type gumshoos) or fishious rend

In [228]:
movetable.loc[['Aura Sphere','Dark Pulse', 'Dragon Pulse'],'Pulse']='Yes' # maybe I can replace this 'Yes' with binary values later using get_dummies?
# haven't included water pulse, heal pulse, or terrain pulse.

In [229]:
movetable.loc[['Psychic Noise','Snarl'],'Sound']='Yes'
# Haven't included a bunch of sound based moves which can't be learned by Toxtricity (punk rock)
# like bug buzz, alluring voice
# also haven't included overdrive and Boomburst

In [230]:
movetable.loc[['Bounce','Dig','Dive','Meteor Beam','Fly','Sky Attack', 'Solar Blade','Solar Beam'],'Two_Turn']='Yes'
# idk how to feel about some of these because escaping a move is nice when you're faster. 
# also meteor beam is too good to be ignored by something like this.

# It still requires power herb, true. Maybe I'll just give it 180 power and call it a day (I did give it 180 power, change it if it feels wrong)

In [231]:
movetable

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Move,,,,,,,,,,,,
Acrobatics,Flying,NaN,None,NaN,None,None,None,None,None,None,None,None
Aerial Ace,Flying,NaN,None,NaN,None,None,None,None,Slicing,None,None,None
Air Cutter,Flying,NaN,None,NaN,None,None,None,None,Slicing,None,None,None
Air Slash,Flying,NaN,None,NaN,None,None,None,None,Slicing,None,None,None
Alluring Voice,Fairy,NaN,None,NaN,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Charge,Electric,NaN,None,NaN,None,None,None,None,None,None,None,None
Wood Hammer,Grass,NaN,None,NaN,None,None,None,None,None,None,None,None
X Scissor,Bug,NaN,None,NaN,None,None,None,None,Slicing,None,None,None


In [232]:
movetable.loc['Acrobatics',['Power','Category']]=[110,'Physical']
movetable.loc['Aerial Ace',['Power','Category']]=[60,'Physical']
movetable.loc['Air Slash',['Power','Category','Accuracy']]=[75,'Special',95]
movetable.loc['Air Cutter',['Power','Category','Accuracy']]=[60,'Special',95]
movetable.loc['Alluring Voice',['Power','Category']]=[80,'Special']
movetable.loc['Aqua Cutter',['Power','Category']]=[70,'Physical']
movetable.loc['Aqua Jet',['Power','Category']]=[40,'Physical']
movetable.loc['Aqua Tail',['Power','Category','Accuracy']]=[90,'Physical',90]
movetable.loc['Aura Sphere',['Power','Category']]=[80,'Special']
movetable.loc['Aurora Beam',['Power','Category']]=[65,'Special']
movetable.loc['Avalanche',['Power','Category']]=[120,'Physical']
movetable.loc['Axe Kick',['Power','Category','Accuracy']]=[120,'Physical',90]
movetable.loc['Belch',['Power','Category','Accuracy']]=[120,'Special',90]
movetable.loc['Bite',['Power','Category']]=[60,'Physical']
movetable.loc['Blaze Kick',['Power','Category','Accuracy']]=[85,'Physical',90]
movetable.loc['Body Press',['Power']]=[80]
movetable.loc['Bone Club',['Power','Category','Accuracy']]=[65,'Physical',85]
movetable.loc['Bone Rush',['Category','Accuracy']]=['Physical',90]
movetable.loc['Bonemerang',['Power','Category','Accuracy']]=[100,'Physical',90]
movetable.loc['Boomburst',['Power','Category']]=[140,'Special']
movetable.loc['Bounce',['Power','Category','Accuracy']]=[85,'Physical',85]
movetable.loc['Brave Bird',['Power','Category']]=[120,'Physical']
movetable.loc['Breaking Swipe',['Power','Category']]=[60,'Physical']
movetable.loc['Brick Break',['Power','Category']]=[75,'Physical']
movetable.loc['Brine',['Power','Category']]=[65,'Special']
movetable.loc['Brutal Swing',['Power','Category']]=[60,'Physical']
movetable.loc['Bubble Beam',['Power','Category']]=[65,'Special']
movetable.loc['Bug Buzz',['Power','Category']]=[90,'Special']
movetable.loc['Bulldoze',['Power','Category']]=[60,'Physical']
movetable.loc['Bullet Punch',['Power','Category']]=[40,'Physical']
movetable.loc['Bullet Seed',['Category']]=['Physical']
movetable.loc['Burning Jealousy',['Power','Category']]=[70,'Special']
movetable.loc['Chilling Water',['Power','Category']]=[50,'Special']
movetable.loc['Close Combat',['Power','Category']]=[120,'Physical']
movetable.loc['Crabhammer',['Power','Category','Accuracy']]=[100,'Physical',90]
movetable.loc['Cross Chop',['Power','Category','Accuracy']]=[100,'Physical',80]
movetable.loc['Cross Poison',['Power','Category']]=[70,'Physical']
movetable.loc['Crunch',['Power','Category']]=[80,'Physical']
movetable.loc['Dark Pulse',['Power','Category']]=[80,'Special']
movetable.loc['Darkest Lairat',['Power','Category']]=[85,'Physical']
movetable.loc['Dazzling Gleam',['Power','Category']]=[80,'Special']
movetable.loc['Dig',['Power','Category']]=[80,'Physical']
movetable.loc['Discharge',['Power','Category']]=[80,'Special']
movetable.loc['Dive',['Power','Category']]=[80,'Physical']
movetable.loc['Draco Meteor',['Power','Category','Accuracy']]=[140,'Special',90]
movetable.loc['Dragon Claw',['Power','Category']]=[80,'Physical']
movetable.loc['Dragon Hammer',['Power','Category']]=[90,'Physical']
movetable.loc['Dragon Pulse',['Power','Category']]=[85,'Special']
movetable.loc['Dragon Rush',['Power','Category','Accuracy']]=[100,'Physical',75]
movetable.loc['Drain Punch',['Power','Category']]=[75,'Physical']
movetable.loc['Draining Kiss',['Power','Category']]=[50,'Special']
movetable.loc['Drill Peck',['Power','Category']]=[80,'Physical']
movetable.loc['Drill Run',['Power','Category','Accuracy']]=[80,'Physical',95]
movetable.loc['Dual Chop',['Power','Category','Accuracy']]=[80,'Physical',90]
movetable.loc['Dual Wingbeat',['Power','Category','Accuracy']]=[80,'Physical',90]
movetable.loc['Dynamic Punch',['Power','Category','Accuracy']]=[80,'Physical',50]
movetable.loc['Earth Power',['Power','Category']]=[90,'Special']
movetable.loc['Earthquake',['Power','Category']]=[100,'Physical']
movetable.loc['Energy Ball',['Power','Category']]=[90,'Special']
movetable.loc['Expanding Force',['Power','Category']]=[80,'Special']
movetable.loc['Extrasensory',['Power','Category']]=[80,'Special']
movetable.loc['Fire Fang',['Power','Category','Accuracy']]=[65,'Physical',95]
movetable.loc['Fire Punch',['Power','Category']]=[75,'Physical']
movetable.loc['First Impression',['Power','Category']]=[90,'Physical']
movetable.loc['Flame Burst',['Power','Category']]=[70,'Special']
movetable.loc['Flamethrower',['Power','Category']]=[90,'Special']
movetable.loc['Flare Blitz',['Power','Category']]=[120,'Physical']
movetable.loc['Flash Cannon',['Power','Category']]=[80,'Special']
movetable.loc['Fling',['Category']]=['Physical']
movetable.loc['Flip Turn',['Power','Category']]=[60,'Physical']
movetable.loc['Fly',['Power','Category','Accuracy']]=[90,'Physical',95]
movetable.loc['Focus Blast',['Power','Category','Accuracy']]=[120,'Special',70]
movetable.loc['Foul Play',['Power']]=[95]
movetable.loc['Freeze Dry',['Power','Category']]=[70,'Special']
movetable.loc['Future Sight',['Power','Category']]=[120,'Special']
movetable.loc['Fury Cutter',['Power','Category','Accuracy']]=[10,'Physical',95]
movetable.loc['Giga Drain',['Power','Category']]=[75,'Special']
movetable.loc['Grass Knot',['Category']]=['Special']
movetable.loc['Gunk Shot',['Power','Category','Accuracy']]=[120,'Physical',80]
movetable.loc['Gyro Ball',['Category']]=['Physical']
movetable.loc['Hammer Arm',['Power','Category','Accuracy']]=[100,'Physical',90]
movetable.loc['Hard Press',['Category']]=['Physical']
movetable.loc['Head Smash',['Power','Category','Accuracy']]=[150,'Physical',80]
movetable.loc['Headlong Rush',['Power','Category']]=[120,'Physical']
movetable.loc['Heat Crash',['Category']]=['Physical']
movetable.loc['Heat Wave',['Power','Category','Accuracy']]=[95,'Special',90]
movetable.loc['Heavy Slam',['Category']]=['Physical']
movetable.loc['Hex',['Power','Category']]=[65,'Special']
movetable.loc['High Horsepower',['Power','Category','Accuracy']]=[95,'Physical',95]
movetable.loc['High Jump Kick',['Power','Category','Accuracy']]=[130,'Physical',90]
movetable.loc['Hurricane',['Power','Category','Accuracy']]=[110,'Special',70]
movetable.loc['Hydro Pump',['Power','Category','Accuracy']]=[110,'Special',80]
movetable.loc['Ice Beam',['Power','Category']]=[90,'Special']
movetable.loc['Ice Fang',['Power','Category','Accuracy']]=[65,'Physical',95]
movetable.loc['Ice Hammer',['Power','Category']]=[100,'Physical']
movetable.loc['Ice Punch',['Power','Category']]=[75,'Physical']
movetable.loc['Ice Shard',['Power','Category']]=[40,'Physical']
movetable.loc['Ice Spinner',['Power','Category']]=[80,'Physical']
movetable.loc['Icicle Spear',['Category']]=['Physical']
movetable.loc['Inferno',['Power','Category','Accuracy']]=[100,'Special',50]
movetable.loc['Iron Head',['Power','Category']]=[80,'Physical']
movetable.loc['Iron Tail',['Power','Category','Accuracy']]=[100,'Physical',75]
movetable.loc['Jaw Lock',['Power','Category']]=[80,'Physical']
movetable.loc['Jump Kick',['Power','Category','Accuracy']]=[100,'Physical',95]
movetable.loc['Knock Off',['Power','Category']]=[97,'Physical']
movetable.loc['Lash Out',['Power','Category']]=[75,'Physical']
movetable.loc['Leaf Blade',['Power','Category']]=[90,'Physical']
movetable.loc['Leech Life',['Power','Category']]=[80,'Physical']
movetable.loc['Liquidation',['Power','Category']]=[85,'Physical']
movetable.loc['Low Kick',['Category']]=['Physical']
movetable.loc['Low Sweep',['Power','Category']]=[65,'Physical']
movetable.loc['Lunge',['Power','Category']]=[80,'Physical']
movetable.loc['Mach Punch',['Power','Category']]=[40,'Physical']
movetable.loc['Megahorn',['Power','Category','Accuracy']]=[120,'Physical',85]
movetable.loc['Meteor Beam',['Power','Category','Accuracy']]=[180,'Special',90]
movetable.loc['Meteor Mash',['Power','Category','Accuracy']]=[90,'Physical',90]
movetable.loc['Mirror Shot',['Power','Category','Accuracy']]=[65,'Special',85]
movetable.loc['Misty Explosion',['Power','Category']]=[100,'Special']
movetable.loc['Moonblast',['Power','Category']]=[95,'Special']
movetable.loc['Mud Bomb',['Power','Category','Accuracy']]=[65,'Special',85]
movetable.loc['Muddy Water',['Power','Category','Accuracy']]=[90,'Special',85]
movetable.loc['Mystical Fire',['Power','Category']]=[75,'Special']
movetable.loc['Night Slash',['Power','Category']]=[70,'Physical']
movetable.loc['Octazooka',['Power','Category','Accuracy']]=[65,'Special',85]
movetable.loc['Outrage',['Power','Category']]=[120,'Physical']
movetable.loc['Overheat',['Power','Category','Accuracy']]=[130,'Special',90]
movetable.loc['Payback',['Power','Category']]=[100,'Physical']
movetable.loc['Petal Blizzard',['Power','Category']]=[90,'Physical']
movetable.loc['Petal Dance',['Power','Category']]=[120,'Special']
movetable.loc['Pin Missile',['Category','Accuracy']]=['Physical',95]
movetable.loc['Play Rough',['Power','Category','Accuracy']]=[90,'Physical',90]
movetable.loc['Poison Fang',['Power','Category']]=[50,'Physical']
movetable.loc['Poison Jab',['Power','Category']]=[80,'Physical']
movetable.loc['Pollen Puff',['Power','Category']]=[90,'Special']
movetable.loc['Poltergeist',['Power','Category','Accuracy']]=[110,'Physical',90]
movetable.loc['Pounce',['Power','Category']]=[50,'Physical']
movetable.loc['Power Gem',['Power','Category']]=[80,'Special']
movetable.loc['Power Trip',['Category']]=['Physical']
movetable.loc['Power Whip',['Power','Category','Accuracy']]=[120,'Physical',85]
movetable.loc['Psybeam',['Power','Category']]=[65,'Special']
movetable.loc['Psychic',['Power','Category']]=[90,'Special']
movetable.loc['Psychic Fangs',['Power','Category']]=[85,'Physical']
movetable.loc['Psychic Noise',['Power','Category']]=[75,'Special']
movetable.loc['Psycho Cut',['Power','Category']]=[70,'Physical']
movetable.loc['Psyshock',['Power','Category']]=[80,'Special']
movetable.loc['Pursuit',['Category']]=['Physical']
movetable.loc['Razor Shell',['Power','Category','Accuracy']]=[75,'Physical',95]
movetable.loc['Revenge',['Power','Category']]=[120,'Physical']
movetable.loc['Reversal',['Power','Category']]=[200,'Physical']
movetable.loc['Rock Blast',['Category','Accuracy']]=['Physical',90]
movetable.loc['Rock Slide',['Power','Category','Accuracy']]=[75,'Physical',90]
movetable.loc['Rock Tomb',['Power','Category','Accuracy']]=[60,'Physical',95]
movetable.loc['Sacred Sword',['Power','Category']]=[90,'Physical']
movetable.loc['Scald',['Power','Category']]=[80,'Special']
movetable.loc['Scale Shot',['Category','Accuracy']]=['Physical',90]
movetable.loc['Scorching Sands',['Power','Category']]=[70,'Special']
movetable.loc['Seed Bomb',['Power','Category']]=[90,'Physical']
movetable.loc['Shadow Ball',['Power','Category']]=[80,'Special']
movetable.loc['Shadow Claw',['Power','Category']]=[70,'Physical']
movetable.loc['Shadow Sneak',['Power','Category']]=[40,'Physical']
movetable.loc['Shock Wave',['Power','Category']]=[60,'Special']
movetable.loc['Signal Beam',['Power','Category']]=[75,'Special']
movetable.loc['Skitter Smack',['Power','Category','Accuracy']]=[70,'Physical',90]
movetable.loc['Sky Attack',['Power','Category','Accuracy']]=[140,'Physical',90]
movetable.loc['Sky Uppercut',['Power','Category','Accuracy']]=[85,'Physical',90]
movetable.loc['Sludge',['Power','Category']]=[65,'Special']
movetable.loc['Sludge Bomb',['Power','Category']]=[90,'Special']
movetable.loc['Sludge Wave',['Power','Category']]=[95,'Special']
movetable.loc['Smack Down',['Power','Category']]=[50,'Physical']
movetable.loc['Smart Strike',['Power','Category']]=[70,'Physical']
movetable.loc['Snarl',['Power','Category','Accuracy']]=[55,'Special',95]
movetable.loc['Solar Beam',['Power','Category']]=[120,'Special']
movetable.loc['Solar Blade',['Power','Category']]=[125,'Physical']
movetable.loc['Spark',['Power','Category']]=[65,'Physical']
movetable.loc['Steamroller',['Power','Category']]=[65,'Physical']
movetable.loc['Steel Wing',['Power','Category','Accuracy']]=[70,'Physical',90]
movetable.loc['Stomping Tantrum',['Power','Category']]=[75,'Physical']
movetable.loc['Stone Edge',['Power','Category','Accuracy']]=[100,'Physical',80]
movetable.loc['Stored Power',['Category']]=['Special']
movetable.loc['Storm Throw',['Power','Category']]=[60,'Physical']
movetable.loc['Submission',['Power','Category','Accuracy']]=[80,'Physical',80]
movetable.loc['Sucker Punch',['Power','Category']]=[70,'Physical']
movetable.loc['Supercell Slam',['Power','Category','Accuracy']]=[100,'Physical',95]
movetable.loc['Superpower',['Power','Category']]=[120,'Physical']
movetable.loc['Surf',['Power','Category']]=[90,'Special']
movetable.loc['Synchronoise',['Power','Category']]=[120,'Special']
movetable.loc['Temper Flare',['Power','Category']]=[75,'Physical']
movetable.loc['Thief',['Power','Category']]=[60,'Physial']
movetable.loc['Throat Chop',['Power','Category']]=[80,'Physical']
movetable.loc['Thunder',['Power','Category','Accuracy']]=[110,'Special',70]
movetable.loc['Thunder Fang',['Power','Category','Accuracy']]=[65,'Physical',95]
movetable.loc['Thunder Punch',['Power','Category']]=[75,'Physical']
movetable.loc['Thunderbolt',['Power','Category']]=[90,'Special']
movetable.loc['Trailblaze',['Power','Category']]=[50,'Physical']
movetable.loc['Triple Axel',['Category','Accuracy']]=['Physical',90]
movetable.loc['Vacuum Wave',['Power','Category']]=[40,'Special']
movetable.loc['Venoshock',['Category']]=['Special']
movetable.loc['Vital Throw',['Power','Category']]=[70,'Physical']
movetable.loc['Wake Up Slap',['Power','Category']]=[70,'Physical']
movetable.loc['Water Shuriken',['Category']]=['Special']
movetable.loc['Waterfall',['Power','Category']]=[80,'Physical']
movetable.loc['Wild Charge',['Power','Category']]=[90,'Physical']
movetable.loc['Wood Hammer',['Power','Category']]=[120,'Physical']
movetable.loc['X Scissor',['Power','Category']]=[80,'Physical']
movetable.loc['Zap Cannon',['Power','Category','Accuracy']]=[120,'Special',50]
movetable.loc['Zen Headbutt',['Power','Category','Accuracy']]=[80,'Physical',90]



# excluded moves currently (I'll delay the base powers of these moves, but will fill in the types of most of these):
# I'll delay filling in Beat Up too
# acrobatics, avalanche, beat up (60-90), Body Press, bone rush, bullet seed, comeuppance, counter, fling, foul play, fury cutter, Grass Knot,
# gyro ball, hard press, heat crash, heavy slam, icicle spear, low kick, metal burst, mirror coat, payback, pin missile, power trip, 
# pursuit, revenge, rock blast, scale shot, stored power,
# triple axel, uturn, venoshock, volt switch, Water Shuriken,
# I've filled in Psyshock, but if you end up doing defensive things later, be mindful
# I'm filling body press and foul play, but am not stating their categories.
# Triple axel has a power of 94 based on this https://bulbapedia.bulbagarden.net/wiki/Triple_Axel_(move), but I haven't filled this yet

# okay fury cutter, I gave 10 BP. 
# I'm not going to fill in the categories of volt switch and uturn.
# most nonstab mons with avalanche are slow, so I'll give it 120 BP
# most nonstab mons with payback have some other good dark move, so I don't mind making it 100
# same goes for revenge. The only mons which depend on Revenge are runerigus and stunfisk which are slow as shit anyway.



# I added bonemerang, dual chop, dual wingbeat even though they hit twice, I should probably do the same for regular multi strike moves.
# maybe I should have a filter for good things against sash/sturdy (including priority moves)?


# there are actually a bunch of moves which become buffed with some condition, like brine, hex, venoshock, acrobatics. I should think about something.
# belch works only when a condition is met.
# acrobatics condition is easy, so I've already given it 110 power. Maybe I should exclude it from this move list altogether.


# when you run this code, make sure to print movetable again. If a move has been spelled incorrectly, it will appear at the bottom as a 'new row'.
movetable

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Move,,,,,,,,,,,,
Acrobatics,Flying,110.0,Physical,NaN,None,None,None,None,None,None,None,None
Aerial Ace,Flying,60.0,Physical,NaN,None,None,None,None,Slicing,None,None,None
Air Cutter,Flying,60.0,Special,95.0,None,None,None,None,Slicing,None,None,None
Air Slash,Flying,75.0,Special,95.0,None,None,None,None,Slicing,None,None,None
Alluring Voice,Fairy,80.0,Special,NaN,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Charge,Electric,90.0,Physical,NaN,None,None,None,None,None,None,None,None
Wood Hammer,Grass,120.0,Physical,NaN,None,None,None,None,None,None,None,None
X Scissor,Bug,80.0,Physical,NaN,None,None,None,None,Slicing,None,None,None


In [233]:
movetable['Accuracy']=movetable['Accuracy'].fillna(100)

In [234]:
#movetable.info()

In [235]:
string_columns=movetable.select_dtypes(include=['object']).columns
movetable[string_columns]=movetable[string_columns].astype('category')
movetable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 213 entries, Acrobatics to Zen Headbutt
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Type                 213 non-null    category
 1   Power                187 non-null    float64 
 2   Category             204 non-null    category
 3   Accuracy             213 non-null    float64 
 4   Two_Turn             8 non-null      category
 5   Slow                 0 non-null      category
 6   Multi_Strike         0 non-null      category
 7   Sheer Force Boosted  0 non-null      category
 8   Contact              28 non-null     category
 9   Pulse                3 non-null      category
 10  Sound                2 non-null      category
 11  Priority             0 non-null      category
dtypes: category(10), float64(2)
memory usage: 16.9+ KB


Creating a dictionary of the moves, which makes it easier to apply as a function

In [236]:
#move_dict=movetable.to_dict() #if I directly convert movetable to dictionary, it makes a nested dictionary with a random single key named 'Type'.
#And the value of this key 'type' is the whole actual dictionary nested as a 'value'

#Dataframes cannot be converted into dictionaries. Only series can be converted into dictionaries.
#movetable['Type'] is a series with Move as an index and Type as values, and hence converting it into a dictionary is straightforward.


move_dict=movetable['Type'].to_dict()


# I think it's because normally dictionaries have the column names as keys, and each key stores every cell below it in a list of 'values'.
# Storing indexes as keys and a single cell as its value is unnatural, which is why we needed to restrict it to only one column.

# There is something called orient='index', which can be looked into. pd.DataFrame.from_dict(d, orient = 'index')

move_dict

{'Acrobatics': 'Flying',
 'Aerial Ace': 'Flying',
 'Air Cutter': 'Flying',
 'Air Slash': 'Flying',
 'Alluring Voice': 'Fairy',
 'Aqua Cutter': 'Water',
 'Aqua Jet': 'Water',
 'Aqua Tail': 'Water',
 'Aura Sphere': 'Fighting',
 'Aurora Beam': 'Ice',
 'Avalanche': 'Ice',
 'Axe Kick': 'Fighting',
 'Beat Up': 'Dark',
 'Belch': 'Poison',
 'Bite': 'Dark',
 'Blaze Kick': 'Fire',
 'Body Press': 'Fighting',
 'Bone Club': 'Ground',
 'Bone Rush': 'Ground',
 'Bonemerang': 'Ground',
 'Boomburst': 'Normal',
 'Bounce': 'Flying',
 'Brave Bird': 'Flying',
 'Breaking Swipe': 'Dragon',
 'Brick Break': 'Fighting',
 'Brine': 'Water',
 'Brutal Swing': 'Dark',
 'Bubble Beam': 'Water',
 'Bug Buzz': 'Bug',
 'Bulldoze': 'Ground',
 'Bullet Punch': 'Steel',
 'Bullet Seed': 'Grass',
 'Burning Jealousy': 'Fire',
 'Chilling Water': 'Water',
 'Close Combat': 'Fighting',
 'Comeuppance': 'Typeless',
 'Counter': 'Typeless',
 'Crabhammer': 'Water',
 'Cross Chop': 'Fighting',
 'Cross Poison': 'Poison',
 'Crunch': 'Dark',
 

# Now that we have a bunch of tables, we can check data accuracy

## Step 1
To see *if* there are errors

Checking for Type errors
Ex: Sometimes Counter may take the type Fighting, and sometimes it may take the type Typeless.

In [237]:
movetable[movetable.index.duplicated(keep=False)]  #keep = false ensures that both the original and the duplicate are shown. (so that we know what the right one is) (because otherwise, if it just shows Pollen Puff: Bug, you won't realize that the duplicate was Pollen Puff with a Null value)
# If Nothing came up here, that's good.
# Right now, the move bulldoze appears to have the type earthquake, so I move on to Step 2.
#This is a duplicate of somewhere else where the move Bulldoze has the type Ground.
# In case something has a blank type, that should also appear here. (As long as it's not a move that appears only once in the whole dataset lol)

,Type,Power,Category,Accuracy,Two_Turn,Slow,Multi_Strike,Sheer Force Boosted,Contact,Pulse,Sound,Priority
Move,,,,,,,,,,,,


Checking for spelling mistakes:

In [238]:
# Printing the moves column and checking it manually for a spelling mistake is a terrible idea I once followed.
# I'll create code which finds the rarest moves. Moves that have appeared less than 5 times.
# Some move spelled like 'Sald' is likely to be a rare move and appear in this 'rare' list.
# And I'll manually check these moves to see if they are genuinely rare moves like Octazooka, or if they are misspellings like Sald.

This won't work, because something really pops up to my eye only when it's directly above or below its actual move name.
So I shall comment all the involved code (which actually worked).

In [239]:
for i in movetable.index:
    print(i)
#movetable.reset_index().drop(columns='Type')

# VIEW OUTPUT AS A SCROLLABLE ELEMENT AND CHECK IF THERE ARE ANY MISSPELLINGS
# (There won't be duplicates because these will have been taken care of in the first code block of step 1.)
# But something like Acrbtics might be above Acrobatics.


Acrobatics
Aerial Ace
Air Cutter
Air Slash
Alluring Voice
Aqua Cutter
Aqua Jet
Aqua Tail
Aura Sphere
Aurora Beam
Avalanche
Axe Kick
Beat Up
Belch
Bite
Blaze Kick
Body Press
Bone Club
Bone Rush
Bonemerang
Boomburst
Bounce
Brave Bird
Breaking Swipe
Brick Break
Brine
Brutal Swing
Bubble Beam
Bug Buzz
Bulldoze
Bullet Punch
Bullet Seed
Burning Jealousy
Chilling Water
Close Combat
Comeuppance
Counter
Crabhammer
Cross Chop
Cross Poison
Crunch
Dark Pulse
Darkest Lairat
Dazzling Gleam
Dig
Discharge
Dive
Draco Meteor
Dragon Claw
Dragon Hammer
Dragon Pulse
Dragon Rush
Drain Punch
Draining Kiss
Drill Peck
Drill Run
Dual Chop
Dual Wingbeat
Dynamic Punch
Earth Power
Earthquake
Energy Ball
Expanding Force
Extrasensory
Fire Fang
Fire Punch
First Impression
Flame Burst
Flamethrower
Flare Blitz
Flash Cannon
Fling
Flip Turn
Fly
Focus Blast
Foul Play
Freeze Dry
Fury Cutter
Future Sight
Giga Drain
Grass Knot
Gunk Shot
Gyro Ball
Hammer Arm
Hard Press
Head Smash
Headlong Rush
Heat Crash
Heat Wave
Heavy Slam


The second step is to see *where in the dataframe* these errors are, so that we can go to the spreadsheet and fix it.

### Code to find where certain moves (or types) have been misspelled a particular way:

Also works for when you've filled a move as 'Dark' or 'Shadow'

In [240]:
#Apparently two of the moves are named 'Countr', which means I messed up somwehere during input,
# so I'm trying to find the index of the places I messed up so that I can edit it in Excel


indices=np.where(df=='Countr')

#print(indices)

row_indices, col_indices = indices[0], indices[1]
print(row_indices,col_indices)

# indices start from 0, and moreover, one row is used up to write the names of the columns
# so an x value of 104 actually corresponds to row 106 in excel.



if len(row_indices)*len(col_indices)>0: #they have to exist otherwise the following line will error due to empty lists
    print(df.iloc[row_indices[0],col_indices[0]]) # this code prints out the numerical index.
    # the following for loop prints out the pokemon name and the column name instead, which are more useful indexes.



for x,y in zip(row_indices, col_indices):
    print(df.loc[x,'Name'],df.columns[y]) #prints the positions of the errors in terms of pokemon name and move column


[] []


Remember to code something later that finds out how many types of moves a mon has (not coverage they apply on, but simply how many types of moves it has. The mons with less types of moves are harder to use in the gauntlet eventually, they'll be rarely picked by the code. So in the chance that they are picked by the code, highly prioritize using  them (like articuno). So try to sort them super high)


You can call this, "Niche pokemon for whom these 4 types are the perfect niche"


### A way to hunt down where there's a mismatch between the move name and the type it's given:

In [241]:
#Helps for when a move sometimes has a weird type nex to it, or a type has a weird move next to it.
# This prints all the indices where Counter is found. And the indices where Typeless is found.


# Maybe I can comment this code now instead of letting it run every time.


indices=np.where(df=="Counter")
row_indices_m, col_indices_m = indices[0], indices[1]
#print(col_indices_m)

indices=np.where(df=="Typeless") #Be sure to replace the word with the desired word later in this cell!
row_indices_t, col_indices_t = indices[0], indices[1]
#print(col_indices_t)

#There is an inequality between the number of times Counter is found and the number of times Typeless is found. 
#So make sure to use both indexes.

print('')


# I'm trying to find out where Counter is found and the corresponding type is not typeless.
# Because my list tells me that there's one particular Counter which claims to be fighting type. (I know I could do == fighting instead of != typeless, but whatever)
# Since I solved the issue, the following code no longer has any output.

#I also used this code to solve an issue where there was a Uturn which wasn't followed by Bug (It was blank). So !="Bug" worked out perfectly.


for x,y in zip(row_indices_m, col_indices_m):
    if df.iloc[x,y+1] != "Typeless":

        print(x,y+1)    # this is the numerical index

        print(df.loc[x,'Name'],df.columns[y+1]) #prints the position of the non-typeless counter in terms of pokemon name and type column



# Most of these are 12 because when the most common index for Counter to be found in movesdf is the Move 1 column which is the 12th column right now.


Nothing currently appears as output to the above cell because it only prints something if there's a place where 'Counter' does not correspond to Typeless'.

## Creating the type offense dictionary. Will be used later as a function to convert types into their coverage

In [242]:
#Type Offense dictionary
#mirror coat and counter are typeless
Off={
    'Normal':[],
    'Dragon':['Dragon'],
    'Ghost':['Psychic','Ghost'],
    'Electric':['Water','Flying'],
    'Fairy':['Fighting','Dragon','Dark'],
    'Poison':['Grass','Fairy'],

    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],

    'Fire':['Grass','Ice','Bug','Steel'],
    'Bug':['Grass','Psychic','Dark'],
    
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Freeze Dry':['Water','Grass','Ground','Flying','Dragon'],


    'Psychic':['Fighting','Poison'],
    'Flying':['Grass','Fighting','Bug'],
    'Steel':['Ice','Rock','Fairy'],

    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so I added that

    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Ground':['Fire','Electric','Poison','Rock','Steel','Dragon'],    #ground is good against nearly half dragons so I added that
    
    'Dark':['Psychic','Ghost'],
    'Typeless':[],
    'nan' :[]

}
#notes: freeze dry is sneakily coded as its own type because that makes stab clearer when it is a stab move (currently freeze dry is always a stab move for every pokemon. If, in the future, it isn't, you need to change the whole code)

In [243]:
# Using the dictionary directly is better, converting it into a series was unnecessary.
#offdf=pd.Series(Off) 
# #DataFrame doesn't work because all arrays need to be the same length, Series allows variable length arrays
# Well even if you convert this series into a dataframe now, you won't exactly get every list to split into multiple columns. But it's fine, we don't need that.

In [244]:
# Using the dictionary directly is better, converting it into a series was unnecessary.
# offdf 
# #this has the attacking types as the indexes, so it's easy to access the values of every type's coverage with loc

Now applying the dictionary/series as a function onto the dataframe.

In [245]:
print(Off)

{'Normal': [], 'Dragon': ['Dragon'], 'Ghost': ['Psychic', 'Ghost'], 'Electric': ['Water', 'Flying'], 'Fairy': ['Fighting', 'Dragon', 'Dark'], 'Poison': ['Grass', 'Fairy'], 'Water': ['Fire', 'Ground', 'Rock'], 'Grass': ['Water', 'Ground', 'Rock'], 'Fire': ['Grass', 'Ice', 'Bug', 'Steel'], 'Bug': ['Grass', 'Psychic', 'Dark'], 'Ice': ['Grass', 'Ground', 'Flying', 'Dragon'], 'Freeze Dry': ['Water', 'Grass', 'Ground', 'Flying', 'Dragon'], 'Psychic': ['Fighting', 'Poison'], 'Flying': ['Grass', 'Fighting', 'Bug'], 'Steel': ['Ice', 'Rock', 'Fairy'], 'Rock': ['Fire', 'Ice', 'Flying', 'Bug', 'Dragon'], 'Fighting': ['Normal', 'Ice', 'Rock', 'Dark', 'Steel'], 'Ground': ['Fire', 'Electric', 'Poison', 'Rock', 'Steel', 'Dragon'], 'Dark': ['Psychic', 'Ghost'], 'Typeless': [], 'nan': []}


maybe make a list of slow mons with priority moves later? or just mons with priority moves. So that extreme speed etc can be included

map the moves to the mons, so that maybe later we can just randomly filter the mons with priority moves who generally have coverage against the elite 4

oh wait, maybe we can make a column for mons called, "Priority", and inside that we can put in a list of the priority moves they have, others will be null

so then in our regular filter we can just see good e4 mons, and also particularly see the good e4 mons which have priority

find a way to sort mons such that mons with broaaaaad matching coverage have lower priority, and mons with specific coverage towards the particular e4 are higher priority. 
Cuz like florges has only bug, psychic, grass, and fairy moves, so the moment you get an elite aligned towards these types, we badly want to use florges cuz it won't get much opportunity otherwise


okay I'm deciding on the physical special split by using base stats. Also sorting the mons based on base stats.
I want to give some higher weightage to huge power, iron fist, and sheer force.


### Now that we've found a way to convert type_df into a dataframe of its coverages, I want to resume working with the original dataframe

In [ ]:
core_df=df[[col for col in df.columns if col.startswith('Type') is False]] #selecting everything but Type columns
core_df

,Name,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
0,Abomasnow,Grass,Ice,NaN,132,132,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Absol,Dark,NaN,NaN,150,115,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Accelgor,Bug,NaN,NaN,70,100,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aerodactyl,Rock,Flying,NaN,105,60,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aggron,Steel,Rock,NaN,140,60,NaN,RU,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,Yanmega,Bug,Flying,NaN,76,116,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,Zangoose,Normal,NaN,NaN,115,60,NaN,RU,NaN,NaN,...,Rock Tomb,Payback,Pursuit,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
485,Zarude,Dark,Grass,NaN,120,70,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,Zebstrika,Electric,NaN,NaN,100,80,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
core_df=core_df.set_index('Name')

#### Quick detour to add Freeze dry as a part of the 'stab'

This can't be done in a previous line while creating core_df because of the line core_df=df[la]

In [248]:
f_indices=np.where(core_df=='Freeze Dry')
f_row_indices= f_indices[0] #col_indices = indices[1]
print(f_row_indices) # finds the integer index for every row which has freeze dry


freeze_dry_indexes=core_df.iloc[[i for i in f_row_indices]].index
core_df.iloc[[i for i in f_row_indices]]  # prints the rows corresponding to the indexes

[ 17  18  23  26  93 102 128 166 167 226 279 290 453]


,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,Trash,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,
Arctovish,Water,Ice,NaN,90,80,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arctozolt,Electric,Ice,NaN,100,90,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Articuno,Ice,Flying,NaN,85,95,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aurorus,Rock,Ice,NaN,77,99,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cryogonal,Ice,NaN,NaN,50,95,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Delibird,Ice,Flying,NaN,55,65,NaN,RU,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Eiscue,Ice,NaN,NaN,80,65,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glaceon,Ice,NaN,NaN,60,130,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glalie,Ice,NaN,NaN,120,120,NaN,RU,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [249]:
index_of_prev_column=core_df.columns.get_loc('MonType3')
core_df.insert(index_of_prev_column+1, 'Freeze_Dry_Stab', None)   # inserting an empty column for freeze dry stab.

In [ ]:
core_df.loc[freeze_dry_indexes,'Freeze_Dry_Stab']='Freeze Dry'


# creates a new column (at the end of the dataframe) where only these rows have a value for Freeze_Dry_Stab. The other rows have NULL.
# above comment line is obsolete, I no longer create the column at the end, I create it next to montype3.


# It's better to create a new column for freeze dry rather than adjust it into the existing stabtype columns, like the mostly unused montype3.
# Anyway we are shrinking the columns later, and I'll make sure that the max becomes 3 columns not 4 columns

In [251]:
# I have decided that using np.where is better than this following code because this will require the apply function which is slow.
# and the np.where has been implemented above.

# I need to account for FreezeDry somehow
# I think this code will fail if a non ice type learns freeze dry someday.

# when freeze dry appears in finally filtered, but the montype ice doesn't, that means the elite has is water but no grass, ground, flying, dragon.
# But I want this freeeze dry to be displayed to the user and also want stabscore to be increased by 1.
# maybe the other stab type of the mon is good against the elite, so I just need to add 1 to the stabscore.

#freeze_dry_mask=core_df.loc['Arctozolt'].isin(['Freeze Dry']) # this applies a mask over every element of arctozolt, and replaces them with True/False
#freeze_dry_mask.any() # if one of the elements in the row is now 'True', then the final output of the row will be True.

# further coding this requires the apply function.

# Writing code to sort the moves so that all dark moves are together, fighting moves are together etc
I think I'll make them reverse alphabetical order to keep fling and stuff at the end.

Counting sort would have been nice, but unfortunately I want to use a sorting method that utilizes sorting (because I'm sorting the moves based on the type, so I want to preserve the different dark moves even if they're sorted by the same value Dark)

Because of this complicated link between moves and types, and this low number of moves (max 47), simple sorting methods will actually be faster than efficient ones.

Simple sorting methods which directly use 'swap', instead of index based stuff will be ideal

In [252]:
tosplit=core_df.copy()
part_1=tosplit.loc[:,'MonType1':'Trash']
# splitting from MonType1 to the Trash column is risky unstable code.
# It's better to spilt to everything before Move1 directly.
part_1 = tosplit.loc[:,:tosplit.columns[(tosplit.columns.get_loc('Move1')-1)]]   # finding the index of Move1, and subtracting 1 gives us the previous column.

# Be careful switching to iloc because slicings can work differently and the column before Move1 also gets deleted (even if iloc has 1 less layer of accessing)
# The equivalent code (iloc code) of the previous line is actually this, without the -1, because of python indexing clashing with pandas. part_1 = tosplit.iloc[:,:tosplit.columns.get_loc('Move1')]

part_2=tosplit.loc[:,'Move1':]
# we're keeping part 1 untouched
# every row of part 2 is going to be replaced with the sorted version of the rows
# I'm probably going to convert each row into a list, sort the list, and then convert it back into a series.

How Srijansort works:
This sorting algorithm is not meant to scale. It assumes that the number of types will not scale. It takes this information directly from the Off dictionary.
It also assumes that a mon doesn't learn more than 15 moves of the same type (for reference, mew learns 14 psychic type moves.) I've made fixed size buckets of size 15 for each type.

This algorithm is very space inefficient but quite simple and fast.

I would have liked to use sorting by counting but that becomes complicated because I'm using a secondary order of types instead of the moves themselves. I also want stable sorting. But this algorithm is kinda similar.

This is sort of like a hashmap. The buckets are arranged in reverse alphabetical order, so nan and Water are the first types (nan is useless but whatever)
The first 15 indexes of the 'bucket_combo' list. are reserved for nan moves. The next 15 indexes are reserved for water moves. The next 15 are reserved for Typless moves. Then Steel moves, etc. I consider these buckets.

When the algorithm encounters a move, like a ghost move, it places it in the ghost bucket. Then it appends the index of ghost by 1 (so that the next ghost move gets placed in the next spot)
#nvm this append index by 1 makes it too complicated cuz I may have to create variables for each type. I'll just use the regular hashing technique and store it in the next spot if the current spot is filled.


Later I shrink down the bucket_combo by removing all the zeroes. (Then I add some zeros back to match the length of the dataframe so that I can insert the row smoothly)

In [253]:
list_of_unique_types=list(Off.keys())
list_of_unique_types

['Normal',
 'Dragon',
 'Ghost',
 'Electric',
 'Fairy',
 'Poison',
 'Water',
 'Grass',
 'Fire',
 'Bug',
 'Ice',
 'Freeze Dry',
 'Psychic',
 'Flying',
 'Steel',
 'Rock',
 'Fighting',
 'Ground',
 'Dark',
 'Typeless',
 'nan']

In [254]:
# read this after you read the function below
def hash(hash_index,move_name,buckets_combo):
    
    if buckets_combo[hash_index]!=0:           #if the cell has already been filled with a type
        return hash(hash_index+1,move_name,buckets_combo) # try hashing into the next spot

    else: # buckets_combo_hash[hash_index]==0:    # if the cell hasn't been filled with a type yet
        buckets_combo[hash_index]=move_name  # then fill the move there
        return buckets_combo

In [ ]:
# this function assumes that there's no blank cell 

def SrijanSort(x,buckets_combo_entry):

    buckets_combo=buckets_combo_entry.copy() #without making a copy, this changes the original input list itself (now go to the Srijansort function call)
    
    for move_name in x.dropna():  # There can be way more than 15 null rows which would mess up the code
        move_type=move_dict.get(move_name)
        base_index=list_of_unique_types.index(move_type)   # so for a type like rock, the base index is 4
        #print(move_name,move_type,base_index)
        

        buckets_combo=hash(base_index*15,move_name,buckets_combo)  # all rock moves are fit between 60 and 75 ( plus or minus 1 index, exact accuracy doesn't matter)

    # by this point, all the moves have been put in buckets_combo, but it's a very sparse matrix with a bunch of zeroes.
    shrunk_buckets=[item for item in buckets_combo if item!=0] # transports every item which is not a zero (essentially deleting all zeroes)
    return shrunk_buckets

In [256]:
buckets_combo=[0]*len(Off.keys())*15
#buckets_combo # this creates an array with 20 x 15 zeroes.

In [257]:
#core_df.apply(series_sorter, axis='columns') # this will work on the contents of each column for every row index (every pokemon)

In [258]:
def series_sorter(the_series,buckets_combo,pad_until_length):

    to_pad=SrijanSort(the_series,buckets_combo)

    padded_series = to_pad + [np.nan]*(pad_until_length - len(to_pad))    # A few None's are adde to reach the length of the dataframe
    # this gives the series the remaining length it needs to be able to be added into the dataframe smoothly
    
    padded_series=pd.Series(padded_series)
    return padded_series

In [259]:
print(len(part_2.columns))
pad_until_length=len(part_2.columns)

43


In [260]:
#part_2=SrijanSort(part_2) #let's do this later

#temppp=part_2.apply(series_sorter, args=(buckets_combo, pad_until_length), axis='columns')
#temppp.columns=part_2.columns


part_2=part_2.apply(series_sorter, args=(buckets_combo, pad_until_length), axis='columns', result_type='broadcast')

# This result_type thing is obscure as hell, crazy that I found a use case for it.
# Basically since the function destroys the dataframe and rebuilds a brand new dataframe of multiple Series from, it's clueless about what the original columns were.
# and it makes the default column names 0,1,2,3 etc
# we could create a temp variable as the result as the function on part_2, and then assign temp.columns= part_2.columns
# but this argument of result type preserves the column names without having to do that.


#newcore_df=pd.concat([part_1,part_2])
#replace this newcore with just core

In [261]:
#temppp

In [262]:
core_df = pd.concat([part_1,part_2], axis=1) # merge them side to side

In [263]:
core_df

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Tier,In Wishlist,Used,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,None,132,132,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,None,150,115,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,None,70,100,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,None,105,60,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,None,140,60,NaN,RU,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,None,76,116,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,None,115,60,NaN,RU,NaN,NaN,...,Payback,Pursuit,Power Trip,Counter,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,None,120,70,NaN,RU,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
#sorted_zoro=series_sorter(part_2.loc['Zoroark'],buckets_combo, pad_until_length)
#sorted_zoro

#back when I was checking if my function could work for a particular row

In [291]:
zoro_row=part_2.loc['Zoroark']
a=[move_dict.get(move_name) for move_name in zoro_row.dropna()]
a

['Ghost',
 'Ghost',
 'Ghost',
 'Poison',
 'Grass',
 'Fire',
 'Fire',
 'Bug',
 'Bug',
 'Psychic',
 'Psychic',
 'Flying',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Ground',
 'Typeless']

In [265]:
#tosplit.loc['Zoroark']=['StringA','StringB','StringC',7,5,6,7,8,9,10,11,12,3,4,5,6,7,8,9,20,1,2,3,4,5,6,7,8,9,30,1,2,3,4,5,6,7,8,9,40,1,2,3,4,5,6,7,8,9,50]
# When adding a row like this, the row must have the same length as the number of columns

# okay now that I've coded a bunch of stuff, let's see if I can add a proper row to this.
# part_2.loc['Zoroark']=sorted_zoro #okay this line works well!

### Calculating an Offence Index which I will add to the dataframe
Since this job is better done during preprocessing.

In [266]:
def Offense_index_temp(Physical_attack, Special_attack):

    #weaker=min(Physical_attack, Special_attack)
    stronger=max(Physical_attack, Special_attack)

    #offense_index= ((stronger-weaker)/(stronger+50)) #this was the formula back when the offence index was always made to be positive
    offense_index= ((Physical_attack-Special_attack)/(stronger+50)) # now a negative index corresponds to a special attacker

    return offense_index

# I might consider some use of negative (it can be direct, because x-y becomes negative when y is greater)(just replace the weaker variable), to differentiate a specially strong mon from a physically strong one.


'''
The logic behind this:
subtracting the stronger stat minus the weaker stat (x-y), and filtering out all the mons with a difference greater than, say, 40, isn't enough.
Special Landorus therian being the meta in OU rn is proof that at high stats, full investment in the weaker stat can be viable.
Fully special mega garchomp exists. Heck, even zero special invested mega chomp carries flamethrower.

So it has to be scaled by some value (possibly the stats themselves).
So we coincidentally arrive at the standardization formula, (x-y/y). (Standardization is a little different because y is a significant number, the mean)

In our case, I believe X is actually the more significant number, since it's the normally used stat. So the formula now is (x-y)/x

This is just base stats. As we know, investment can make a huge difference in viability. I don't want to compare the base atk with the base spa
I want to assume a mon is fully invested in its weaker stat for the whole run, and see if it's viable even then (with choice specs if you'd like). 
and compare this against a fully invested stronger stat.

as we know, real stat = 2 * base stat + 5 + 31 + 63
the 5 and 31 come even without investment, but they do make a difference. An addition to both the numerator and denominator of something does make a difference.
for example, if we had just been seeing the physical/special ratio, 100/60 is smaller than 236/156.

anyway back to our problem. real stat = 2 * base stat + 99
when we plug this into our formula, we get ((2x+99)-(2y+99))/(2x+99)
This simplifies into (assuming 99/2=50):
(x + y)/(x+50)

Based on the distribution of a bunch of mons (you can see the list here on SrijanBadhya's notes): https://keep.google.com/u/0/#NOTE/1nMFXKpsEmNTK_RWcE58ARL-WjtuAGqgkp4h3x145J3pFODCK4gfaWYUcD96N2Q
I've chosen to draw the line at 0.24
If the mon has an offence index less than 0.24, that means its atk and spa are close enough to each other. If they are greater than 0.24, they're too far
0.24 is fairly inclusive. running choice specs on a physical mon is quite viable at that point (even if you didn't invest).


Even though tapu lele has 85 attack, not using the 130 SpA is just wasted potential. 
A regular stab move from SpA would usually hit harder than a sup eff move, and even if the sup eff move would hit harder, it's not worth running any nature besides -atk for the whole run


a pokemon with its stronger stat as 100, can't use its weaker stat if its weaker stat is below 64
a pokemon with its stronger stat as 90, can't use its weaker stat if its weaker stat is below 56
' ' 80, ' ' is below 49
'''

"\nThe logic behind this:\nsubtracting the stronger stat minus the weaker stat (x-y), and filtering out all the mons with a difference greater than, say, 40, isn't enough.\nSpecial Landorus therian being the meta in OU rn is proof that at high stats, full investment in the weaker stat can be viable.\nFully special mega garchomp exists. Heck, even zero special invested mega chomp carries flamethrower.\n\nSo it has to be scaled by some value (possibly the stats themselves).\nSo we coincidentally arrive at the standardization formula, (x-y/y). (Standardization is a little different because y is a significant number, the mean)\n\nIn our case, I believe X is actually the more significant number, since it's the normally used stat. So the formula now is (x-y)/x\n\nThis is just base stats. As we know, investment can make a huge difference in viability. I don't want to compare the base atk with the base spa\nI want to assume a mon is fully invested in its weaker stat for the whole run, and see 

In [267]:
#map(lambda x,y: Offense_index_temp(x,y), database_to_be_modified['Atk'], database_to_be_modified['SpA'])

# for some reason this doesn't work if I substitute list() with pd.Series(). It gives NaN values.



offence_indexes= list(
    map(
        lambda x,y: Offense_index_temp(x,y), core_df['Atk'], core_df['SpA']
        )
)
# this function is so simple, I tried so much zip stuff needlessly because I didn't know lambda functions and map could handle 2 inputs.

In [268]:
insertion_index=core_df.columns.get_loc('Abilities')

core_df.insert(insertion_index+1, 'Offense_index', offence_indexes)
core_df

,MonType1,MonType2,MonType3,Freeze_Dry_Stab,Atk,SpA,Abilities,Offense_index,Tier,In Wishlist,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,None,132,132,NaN,0.000000,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,None,150,115,NaN,0.175000,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,None,70,100,NaN,-0.200000,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,None,105,60,NaN,0.290323,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,None,140,60,NaN,0.421053,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,None,76,116,NaN,-0.240964,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,None,115,60,NaN,0.333333,RU,NaN,...,Payback,Pursuit,Power Trip,Counter,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,None,120,70,NaN,0.294118,RU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
import json
with open(r"jsonfiles/Off.json","w") as f:
    json.dump(Off, f)

In [270]:
# the commented code is obsolete now that I shifted from saving just the dictionary as the json file, to saving the whole dataframe as a json file.
'''
import json
with open(r"jsonfiles/move_dict.json","w") as f:
    json.dump(move_dict, f)
'''
movetable.to_json(r"jsonfiles/move_dict_table.json")

# if this gives you an error about index not being unique, then scroll up to the data accuracy checking code

In [271]:
core_df.to_json(r"jsonfiles/core_df.json")